In [1]:
import tensorflow as tf
import keras as kr
from keras.models import Sequential
from tensorflow.keras import layers

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *

In [4]:
import random

In [5]:
from deap import creator, base, tools, algorithms

Загрузка данных

In [6]:
data=pd.read_csv('C:/Users/Vi/Desktop/data.txt')

In [7]:
data=data.drop_duplicates()

функция оценки решения по MSE на валидационной выборке

In [8]:
def evaluate_MO(individual):
    
    layer_number=individual[0][0]
    layer_size=individual[0][1]
    
    model = Sequential()
    model.add(layers.Dense(layer_size,input_dim=11, activation='sigmoid'))
    
    for i in range(layer_number):
        model.add(layers.Dense(layer_size, activation='sigmoid'))
        
    model.add(layers.Dense(1))
    
    model.compile(optimizer=kr.optimizers.Adam(learning_rate=0.001,beta_1=0.9,
    beta_2=0.999,epsilon=1e-07),loss='mean_squared_error')
    
    model.fit(x_train, y_train,epochs=50, batch_size=10, verbose=0)
    
    pred = model.predict(x_testval).flatten() 

    cost_f1 = mean_squared_error(y_testval, pred)
    
    return (cost_f1,)

описание индивида
x1=количество слоев ИНС
x2=размер скрытого слоя

In [9]:
def gen():
    
    x1=random.randint(2, 6)
    x2=random.randint(10, 44)
        
    return [x1,x2]

функция мутации

In [10]:
def mutation(individual):
    
    print(individual)
    
    start=individual[0][1]
    start=start-5
    finish=individual[0][1]+5
    finish=finish+5
    individual[0][1]=random.randint(start, finish)

    return individual,

Заполнение пропусков и удаление повторов

In [11]:
data=data.fillna(value=0)

In [12]:
data=data.drop_duplicates()

выделение обучающей и тестовой выборки

In [13]:
X = data[['скор ветра', 'выс волны', 'угол волны', 'скорость течения', 'направление течения', 'глубина', 'mass',
       'speedafloat', 'draft', 'owerallwidth', 'length']]

Y = data[['speed']]

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.95, random_state=20)

нормализайия данных

In [15]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train -= mean
x_train /= std
x_test -= mean
x_test /= std

выдеоение валидационной выборки

In [16]:
x_trval, x_testval, y_trval, y_testval = train_test_split(x_test, y_test, test_size=0.5, random_state=20)

выбор параметров эволюционного алгоритма

In [17]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("gen", gen)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.gen, n=1)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)



toolbox.register("evaluate", evaluate_MO)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutation)
toolbox.register("select", tools.selTournament)

создание популяции

In [22]:
pop = toolbox.population(n=10)

In [23]:
print(pop)

[[[3, 35]], [[6, 14]], [[2, 34]], [[5, 28]], [[4, 24]], [[2, 16]], [[5, 18]], [[4, 42]], [[2, 41]], [[3, 24]]]


запуск алгоритма

In [24]:
CXPB, MUTPB, NGEN = 0.7, 0.2, 10

# Evaluate the entire population
fitnesses = map(toolbox.evaluate, pop)
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

g=0

while (g <NGEN) :
    print('generation',g)
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop),tournsize=2)
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover and mutation on the offspring
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1[0], child2[0])
            
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    
    fits_1 = [ind.fitness.values[0] for ind in pop]
    print(pop)
    print(fits_1)
    
    g+=1


generation 0
[[4, 42]]
[[[5, 42]], [[4, 18]], [[5, 18]], [[2, 16]], [[4, 46]], [[3, 24]], [[4, 35]], [[3, 24]], [[4, 18]], [[5, 42]]]
[0.7927429829145132, 2.333080513608168, 1.283939180064192, 2.3773717405967894, 0.5164304632570775, 0.7674975667340613, 0.5452875645859157, 1.573337193802819, 1.134697580282673, 0.6828577444493743]
generation 1
[[4, 35]]
[[3, 24]]
[[[4, 46]], [[4, 35]], [[5, 24]], [[3, 42]], [[4, 35]], [[4, 30]], [[3, 20]], [[4, 35]], [[3, 35]], [[4, 24]]]
[1.116036600549287, 1.688312193511267, 1.0725069465151993, 0.722146617117181, 0.7890602496745135, 1.8196678378696147, 1.6687814090886093, 0.5452875645859157, 0.4243844579079881, 1.3744765897450828]
generation 2
[[3, 35]]
[[3, 35]]
[[[3, 35]], [[4, 35]], [[4, 20]], [[3, 35]], [[4, 35]], [[3, 45]], [[5, 35]], [[4, 24]], [[3, 40]], [[3, 42]]]
[0.7919406961828451, 0.41239637858610695, 2.817452999432139, 0.4045437472174003, 0.5452875645859157, 0.398774270661374, 1.8549494036563656, 0.8298888930421894, 0.7150310999364983, 0.7